In [1]:
#!pip install timm
#!pip install ipywidgets

# New model version
hybrid architecture - ConvNeXt

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from datasets import load_from_disk
import timm
import numpy as np
from tqdm.auto import tqdm
from torchvision.transforms import (
    Compose, Resize, CenterCrop, ToTensor, Normalize, 
    RandomHorizontalFlip, RandomResizedCrop
)

DATA_PATH = "processed_bird_data"
OUTPUT_DIR = "convnext_checkpoints"
MODEL_NAME = "convnext_tiny"
BATCH_SIZE = 32
EPOCHS = 25
LEARNING_RATE = 4e-3

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


Applying Aggressive Data Augmentation to prevent overfitting:

1. Does random resize/zoom (scale 0.8–1.0). Forces the model to recognize a bird by looking at its specific feature (e.g. the head, wing, etc.) instead of the background (e.g. tress).
2. Incorporates horizontal flipping to double the training data.

In [3]:
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_transforms = Compose([
    RandomResizedCrop(224, scale=(0.8, 1.0)), 
    RandomHorizontalFlip(),
    ToTensor(),
    normalize,
])

val_transforms = Compose([
    Resize(256),
    CenterCrop(224),
    ToTensor(),
    normalize,
])

def apply_train_transforms(batch):
    batch["pixel_values"] = [train_transforms(img.convert("RGB")) for img in batch["image"]]
    return batch

def apply_val_transforms(batch):
    batch["pixel_values"] = [val_transforms(img.convert("RGB")) for img in batch["image"]]
    return batch

print("Loading and transforming data...")
dataset = load_from_disk(DATA_PATH)

dataset["train"].set_transform(apply_train_transforms)
dataset["validation"].set_transform(apply_val_transforms)

def collate_fn(batch):
    pixel_values = torch.stack([x["pixel_values"] for x in batch])
    labels = torch.tensor([x["label"] for x in batch])
    return pixel_values, labels

train_loader = DataLoader(dataset["train"], batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(dataset["validation"], batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"Train batches: {len(train_loader)} | Val batches: {len(val_loader)}")

Loading and transforming data...
Train batches: 105 | Val batches: 19


# Training
Initializing CoNeXt model with random weights, no pretraining. 

It uses standard Convolutional layers in the early stages to extract low-level features (edges, textures), and then uses Transformer layers in the final stages to understand the global shape of the bird. Also:

1. High learning rate;
2. 15 epochs;
3. Uses regularization to prevent the model from memorizing the training images.

(using TimmTrainer class as the timm library and Hugging Face Trainer have different variable names (pixel_values vs x)).

In [4]:
model = timm.create_model(
    MODEL_NAME, 
    pretrained=False,
    num_classes=200
)
model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.05)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Training accuracy:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    # Progress bar for the batch
    pbar = tqdm(loader, desc="Training", leave=False)
    
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        # Metrics
        total_loss += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
        
        # Update progress bar
        pbar.set_postfix({"loss": loss.item()})
        
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            logits = model(images)
            loss = criterion(logits, labels)
            
            total_loss += loss.item() * images.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)
            
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy


print("\nStarting Training.")
print(f"{'Epoch':<6} | {'Train Loss':<10} | {'Train Acc':<10} | {'Val Loss':<10} | {'Val Acc':<10}")
print("-" * 55)

best_val_acc = 0.0

for epoch in range(1, EPOCHS + 1):
    # Training
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    
    # Validation
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    
    scheduler.step()
    
    # Save the best model:
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "final_convnext_model.pth")
        saved_msg = "(*)"

    # So the best epoch will be saved
    else:
        saved_msg = ""
        
    print(f"{epoch:<6} | {train_loss:<10.4f} | {train_acc*100:<9.2f}% | {val_loss:<10.4f} | {val_acc*100:<9.2f}% {saved_msg}")

print("\nTraining Complete. Best Validation Accuracy: {:.2f}%".format(best_val_acc*100))


Starting Training.
Epoch  | Train Loss | Train Acc  | Val Loss   | Val Acc   
-------------------------------------------------------


Training:   0%|          | 0/105 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/19 [00:00<?, ?it/s]

1      | 5.7056     | 1.08     % | 5.2438     | 0.85     % (*)


Training:   0%|          | 0/105 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/19 [00:00<?, ?it/s]

24     | 3.2684     | 20.47    % | 4.4003     | 7.64     % 


Training:   0%|          | 0/105 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/19 [00:00<?, ?it/s]

25     | 3.2458     | 20.98    % | 4.4075     | 7.30     % 

Training Complete. Best Validation Accuracy: 9.00%
